# Atlanta Crime Report: 2009 - 2022

### Import Libraries

In [1]:
import pandas as pd
import altair as alt
import os
from csv_parser import csv_columnNames_to_rows
import datetime

### Specify target folder

In [2]:
dirname = 'COBRA-Data'

### View column names of each dataFrame
Wrote a script to create a dataFrame out of the column names for each CSV file in a given folder.  
https://www.linkedin.com/pulse/csv-column-name-parser-brandon-wilson/

In [3]:
csv_columnNames_to_rows(dirname)

Importing: ['COBRA-2009-2019.csv', 'COBRA-2020(NEW RMS 9-30 12-31).csv', 'COBRA-2020-OldRMS-09292020.csv', 'COBRA-2021.csv', 'COBRA-2022.csv'] 
...
loading: COBRA-Data/COBRA-2009-2019.csv
loading: COBRA-Data/COBRA-2020(NEW RMS 9-30 12-31).csv
loading: COBRA-Data/COBRA-2020-OldRMS-09292020.csv
loading: COBRA-Data/COBRA-2021.csv
loading: COBRA-Data/COBRA-2022.csv
... 
Success


,COBRA-2009-2019.csv,COBRA-2020(NEW RMS 9-30 12-31).csv,COBRA-2020-OldRMS-09292020.csv,COBRA-2021.csv,COBRA-2022.csv
0,Report Number,offense_id,offense_id,offense_id,offense_id
1,Report Date,rpt_date,rpt_date,rpt_date,rpt_date
2,Occur Date,occur_date,occur_date,occur_date,occur_date
3,Occur Time,occur_time,occur_time,occur_day,occur_day
4,Possible Date,poss_date,poss_date,occur_day_num,occur_day_num
5,Possible Time,poss_time,poss_time,occur_time,occur_time
6,Beat,beat,beat,poss_date,poss_date
7,Apartment Office Prefix,apt_office_prefix,apartment_office_prefix,poss_time,poss_time
8,Apartment Number,apt_office_num,apartment_number,beat,beat
9,Location,location,location,zone,zone


## Prepare data

In [4]:
dfs = []
c = 0
for file in os.listdir(dirname):
    if c == 0:
        dfs.append(pd.read_csv(dirname + '/' + file, low_memory=False, parse_dates=['Occur Date']))
    else:
        dfs.append(pd.read_csv(dirname + '/' + file, low_memory=False, parse_dates=['occur_date']))
    c+=1

### Keep and rename columns for merge

In [5]:
keep = ['Report Number', 'Occur Date', 'Occur Time', 'Location', 'UCR Literal', 'Neighborhood', 'Latitude', 'Longitude']

dfs[1].rename(columns = {'offense_id':'Report Number', 'rpt_date':'Report Date', 'occur_date':'Occur Date', 
                         'occur_time':'Occur Time', 'location':'Location', 'UC2_Literal':'UCR Literal', 
                          'neighborhood':'Neighborhood', 'lat':'Latitude', 'long':'Longitude'}, inplace = True)

dfs[2].rename(columns = {'offense_id':'Report Number', 'rpt_date':'Report Date', 'occur_date':'Occur Date', 
                         'occur_time':'Occur Time', 'location':'Location', 'UC2_Literal':'UCR Literal', 
                          'neighborhood':'Neighborhood', 'lat':'Latitude', 'long':'Longitude'}, inplace = True)

dfs[3].rename(columns = {'offense_id':'Report Number', 'rpt_date':'Report Date', 'occur_date':'Occur Date', 
                         'occur_time':'Occur Time', 'location':'Location', 'UC2_Literal':'UCR Literal', 
                          'neighborhood':'Neighborhood', 'lat':'Latitude', 'long':'Longitude'}, inplace = True)

dfs[4].rename(columns = {'offense_id':'Report Number', 'rpt_date':'Report Date', 'occur_date':'Occur Date', 
                         'occur_time':'Occur Time', 'location':'Location', 'UC2_Literal':'UCR Literal', 
                          'neighborhood':'Neighborhood', 'lat':'Latitude', 'long':'Longitude'}, inplace = True)

### Drop unwanted columns

In [6]:
# Passes in a list data frames and a list of column names to keep for each data frame
def dfs_dropcols(dfs, keep):
    c = 0
    for df in dfs:
        dfs[c] = df[keep]
        c+=1
    return dfs
dfs = dfs_dropcols(dfs, keep)

In [7]:
# Test drop function
for df in dfs:
    print('df:', list(df))

df: ['Report Number', 'Occur Date', 'Occur Time', 'Location', 'UCR Literal', 'Neighborhood', 'Latitude', 'Longitude']
df: ['Report Number', 'Occur Date', 'Occur Time', 'Location', 'UCR Literal', 'Neighborhood', 'Latitude', 'Longitude']
df: ['Report Number', 'Occur Date', 'Occur Time', 'Location', 'UCR Literal', 'Neighborhood', 'Latitude', 'Longitude']
df: ['Report Number', 'Occur Date', 'Occur Time', 'Location', 'UCR Literal', 'Neighborhood', 'Latitude', 'Longitude']
df: ['Report Number', 'Occur Date', 'Occur Time', 'Location', 'UCR Literal', 'Neighborhood', 'Latitude', 'Longitude']


### Fix Date Formatting and Range

In [8]:
# Checking date formatting
# for df in dfs:
#     print(df['Occur Date'].head())

In [9]:
dfs[1]['Occur Date'] = pd.to_datetime(dfs[1]['Occur Date'], errors = 'coerce')

In [10]:
# Checking date formatting after adjustment
# for df in dfs:
#     print(df['Occur Date'].head())

### Fix Time Formatting

In [11]:
# Checking time formatting
# for df in dfs:
#     print(df['Occur Time'].head())

In [12]:
# fix dfs[0]
dfs[0]['Occur Time']

0         1145
1         1330
2         1500
3         1450
4         1600
          ... 
342909    2030
342910    0432
342911    0920
342912    1853
342913    2045
Name: Occur Time, Length: 342914, dtype: object

In [13]:
lst = []
for time in dfs[0]['Occur Time']:
    lst.append(time[0:2]+':'+time[2:])

In [14]:
dfs[0]['Occur Time'] = lst

In [15]:
dfs[0].head()

,Report Number,Occur Date,Occur Time,Location,UCR Literal,Neighborhood,Latitude,Longitude
0,90010930,2009-01-01,11:45,2841 GREENBRIAR PKWY,LARCENY-NON VEHICLE,Greenbriar,33.68845,-84.49328
1,90011083,2009-01-01,13:30,12 BROAD ST SW,LARCENY-NON VEHICLE,Downtown,33.75320,-84.39201
2,90011208,2009-01-01,15:00,3500 MARTIN L KING JR DR SW,LARCENY-NON VEHICLE,Adamsville,33.75735,-84.50282
3,90011218,2009-01-01,14:50,3393 PEACHTREE RD NE,LARCENY-NON VEHICLE,Lenox,33.84676,-84.36212
4,90011289,2009-01-01,16:00,2841 GREENBRIAR PKWY SW,LARCENY-NON VEHICLE,Greenbriar,33.68677,-84.49773


## Union dfs tables

In [16]:
df_union = None
for df in dfs:
    df_union = pd.concat([df_union, df]).drop_duplicates()
    
# check union 
summ = 0 
c = 0
for df in dfs:
    print(df.shape[0], f'rows in dfs[{c}]')
    summ+=df.shape[0]
    c+=1
print('...')
print(summ, 'rows in dfs total')
print(df_union.shape[0], 'rows in df_union total')
print('...')
print(summ - df_union.shape[0], 'duplicates were removed')

342914 rows in dfs[0]
7249 rows in dfs[1]
14831 rows in dfs[2]
21397 rows in dfs[3]
14605 rows in dfs[4]
...
400996 rows in dfs total
400813 rows in df_union total
...
183 duplicates were removed


### Processing dates 

In [17]:
df_union.sort_values(by = 'Occur Date', ascending = True, inplace = True)  
df_union

,Report Number,Occur Date,Occur Time,Location,UCR Literal,Neighborhood,Latitude,Longitude
240079,160442700,1916-01-07,12:15,710 JEWEL CT SW,BURGLARY-RESIDENCE,Midwest Cascade,33.725830,-84.550500
243237,160902737,1916-03-29,23:00,180 WALKER ST SW,AUTO THEFT,Castleberry Hill,33.749640,-84.401320
243631,160951996,1916-04-02,17:00,2175 PIEDMONT RD NE,BURGLARY-NONRES,Lindridge/Martin Manor,33.817160,-84.366320
247606,161460989,1916-05-15,20:00,102 W PACES FERRY RD NW,LARCENY-NON VEHICLE,Peachtree Heights West,33.841310,-84.384270
248620,161592043,1916-05-30,14:00,4666 EDWINA LN SW,LARCENY-NON VEHICLE,Greenbriar Village,33.703090,-84.540440
...,...,...,...,...,...,...,...,...
19172,213303737,NaT,NaN,"2265 MARIETTA BLVD NW\nATLANTA, GA 30318\nUNIT...",AUTO THEFT,NaN,33.818629,-84.449292
20547,213490677,NaT,NaN,"507 BISHOP ST NW\nATL, GA 30318\nUNITED STATES",LARCENY-FROM VEHICLE,Loring Heights,33.792566,-84.404601
9837,221720624,NaT,NaN,"1823 JONESBORO RD SE\nATLANTA, GA 30315\nUNITE...",LARCENY-NON VEHICLE,Lakewood Heights,33.704411,-84.378159
12347,222081763,NaT,NaN,"1348 BENTEEN WAY SE\nATL, GA 30315\nUNITED STATES",LARCENY-NON VEHICLE,Benteen Park,33.718693,-84.366332


__________________________________

# Analysis

### Time-series analysis
This is a time-series analysis of volume.

In [18]:
# Select columns of interest
keep = ['Report Number', 'Occur Date']
ts_data = df_union[keep]

# Drop rows with NaT and older than 2009
ts_data = ts_data.dropna()
ts_data = ts_data.drop(ts_data[ts_data['Occur Date'] < '2009-01-01'].index)
seasonal_data = ts_data # creating copy of data here to use in another model

# Group by date to track volume each day
ts_data = ts_data.groupby('Occur Date')['Report Number'].count().reset_index() 
ts_data.columns = ['Occur Date', 'Volume']

# Add a 90-day moving average
ts_data['90-day'] = ts_data.rolling(window = 90).mean()

ts_data

,Occur Date,Volume,90-day
0,2009-01-01,133,NaN
1,2009-01-02,144,NaN
2,2009-01-03,126,NaN
3,2009-01-04,95,NaN
4,2009-01-05,126,NaN
...,...,...,...
4987,2022-08-28,54,64.311111
4988,2022-08-29,65,64.322222
4989,2022-08-30,40,64.000000
4990,2022-08-31,49,63.722222


In [19]:
# Build the time-series model

a = alt.Chart(ts_data).mark_area().encode(
    x = 'Occur Date',
    y = 'Volume',
    color = alt.value('#7CB9E8')
).properties(
    width = 800,
   height = 300,
    title = 'Crime Volume Over Time'
).interactive()

r = alt.Chart(ts_data).mark_line().encode(
    x = 'Occur Date',
    y = '90-day',
    color = alt.value('#FF0808'),
    tooltip = ('Volume')
)

time = alt.layer(a, r)

### Seasonal Analysis

In [20]:
#Renaming columns
seasonal_data.rename(columns = {'Occur Date':'Month', 'Report Number':'Volume'}, inplace=True)

# Group by date to track volume each day
seasonal_data = seasonal_data.groupby(seasonal_data['Month'].dt.month)['Volume'].count().reset_index()

#Converting into month name (I did this extra step to avoid weird sorting from group by straight into month name)
seasonal_data['Month'] = pd.to_datetime(seasonal_data['Month'], format='%m').dt.month_name().str.slice(stop=3)

seasonal_data

,Month,Volume
0,Jan,33244
1,Feb,26939
2,Mar,29780
3,Apr,31384
4,May,35001
5,Jun,35301
6,Jul,37203
7,Aug,36348
8,Sep,33019
9,Oct,34325


In [21]:
seasonal = alt.Chart(seasonal_data).mark_bar().encode(
    x = alt.X('Month', sort=list(seasonal_data['Month'])),
    y = 'Volume',
    tooltip = ('Month', 'Volume'),
    color = alt.value('#7CB9E8')
).properties(
    width = 800,
    height = 300,
    title = 'Crime Volume by Month'
)

### Heat Map of Date:Time Volume

In [22]:
keep = ['Occur Date', 'Occur Time', 'Report Number']



heat_data = df_union[keep] #  copy of data here to use in another model
heat_data = heat_data.dropna()
heat_data = heat_data.drop(heat_data[heat_data['Occur Date'] < '2009-01-01'].index).reset_index(drop = True)

lst = []
for m in heat_data['Occur Date']:
    lst.append(m.month)
heat_data['Occur Date'] = lst

lst = []
for t in heat_data['Occur Time']:
    lst.append(t[0:2])
heat_data['Occur Time'] = lst

lst = []
for n in heat_data['Report Number']:
    lst.append(1)
heat_data['Report Number'] = lst

heat_data.rename(columns = {'Occur Date':'Month', 'Occur Time': 'Hour', 'Report Number':'Volume'}, inplace=True)

# Fix Hour formatting
lst = []
for h in heat_data['Hour']:
    if h[1] == ':': 
        lst.append('0'+h[0])
    else:
        lst.append(h)
heat_data['Hour'] = lst
lst = heat_data['Month'] 
sort = lst.unique()

lst = []
for h in heat_data['Hour']:
    if h == '00':
        lst.append('24')
    else:
        lst.append(h)
heat_data['Hour'] = lst

# Drop weird Hour
heat_data = heat_data[heat_data['Hour']!='0T']


# Group by month and hour
heat_data = heat_data.groupby(['Month', 'Hour']).count().reset_index()

# Converting into month name
heat_data['Month'] = pd.to_datetime(heat_data['Month'], format='%m').dt.month_name().str.slice(stop=3)

heat_data

,Month,Hour,Volume
0,Jan,01,1030
1,Jan,02,797
2,Jan,03,605
3,Jan,04,467
4,Jan,05,427
...,...,...,...
283,Dec,20,2110
284,Dec,21,1845
285,Dec,22,1758
286,Dec,23,1537


In [23]:
# heat map
heat = alt.Chart(heat_data).mark_rect().encode(
    x=alt.X('Hour'),
    y=alt.Y('Month', sort= sort),
    color=alt.Color('Volume'),
    tooltip = ('Month', 'Hour', 'Volume')
).properties(
    width = 500,
    height = 500,
    title = 'Crime Volume by Hour/Month'
)

### Neighborhood Analysis

In [24]:
keep = ['Neighborhood', 'Report Number']
neighborhood_data = df_union[keep]
neighborhood_data = neighborhood_data.dropna()
neighborhood_data = neighborhood_data.drop_duplicates()
neighborhood_data = neighborhood_data.groupby(by='Neighborhood').count()
neighborhood_data = neighborhood_data.rename(columns ={'Report Number':'Volume'})
neighborhood_data = neighborhood_data.sort_values(by='Volume', ascending=False).reset_index()
neighborhood_data = neighborhood_data.head(20)

alt.data_transformers.disable_max_rows()
neighborhood = alt.Chart(neighborhood_data).mark_bar().encode(
    x = 'Volume',
    y = alt.Y('Neighborhood', sort = 'x')
).properties(
    width = 400,
    height = 500,
    title = 'Most Dangerous Neighborhoods'
)

### Top Offenses 

In [30]:
topo = df_union['UCR Literal']

topo = topo.to_frame()
topo['Volume'] = 1
topo = topo.groupby(by = 'UCR Literal').sum()
topo = topo.sort_values(by='Volume', ascending = False).reset_index()
topo = topo.head(12)

to = alt.Chart(topo).mark_bar().encode(
    x = alt.X('UCR Literal',sort ='-y'),
    y = 'Volume',
).properties(
    title = 'Top 10 Common Offenses',
    width = 300,
    height = 500
)

# Visualizations

In [51]:
# pip install altair_viewer
import altair as alt

In [52]:
# alt.renderers.enable('mimetype')
# alt.renderers.enable('default')
# alt.renderers.enable('html')
# alt.renderers.enable('notebook')
alt.renderers.enable('altair_viewer')

RendererRegistry.enable('altair_viewer')

In [53]:
time&seasonal

Displaying chart at http://localhost:51662/

In [54]:
neighborhood|to

Displaying chart at http://localhost:51662/

In [55]:
heat 

Displaying chart at http://localhost:51662/